In [ ]:
# reproducable randomness
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)

In [ ]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
from neural_stylization.reconstruct_style import reconstruct_style
from neural_stylization.optimizers import SGD, L_BFGS
from neural_stylization.util.build_callback import build_callback

# SGD

In [ ]:
sgd = reconstruct_style(
    style_path='img/styles/starry_night_google.jpg',
    image_shape=(500, 300),
    optimize=SGD(learning_rate=1e-1),
    iterations=100,
    callback=build_callback('build/style/starry_night/sgd')
)

# L-BFGS

In [ ]:
lbfgs = reconstruct_style(
    style_path='img/styles/starry_night_google.jpg',
    image_shape=(500, 300),
    optimize=L_BFGS(),
    iterations=10,
    callback=build_callback('build/style/starry_night/lbfgs')
)

## Visualization of all 5 Gatys et al. Style Layer Sets

In [ ]:
block1_conv1 = reconstruct_style(
    style_path='img/styles/starry_night_google.jpg',
    layer_names=['block1_conv1'],
    image_shape=(500, 300),
    optimize=L_BFGS(),
    iterations=10,
    callback=build_callback('build/style/block1_conv1')
)
block1_conv1.save('tex/img/style/block1_conv1.png')

In [ ]:
block2_conv1 = reconstruct_style(
    style_path='img/styles/starry_night_google.jpg',
    layer_names=['block1_conv1', 'block2_conv1'],
    image_shape=(500, 300),
    optimize=L_BFGS(),
    iterations=10,
    callback=build_callback('build/style/block2_conv1')
)
block2_conv1.save('tex/img/style/block2_conv1.png')

In [ ]:
block3_conv1 = reconstruct_style(
    style_path='img/styles/starry_night_google.jpg',
    layer_names=['block1_conv1', 'block2_conv1', 'block3_conv1'],
    image_shape=(500, 300),
    optimize=L_BFGS(),
    iterations=10,
    callback=build_callback('build/style/block3_conv1')
)
block3_conv1.save('tex/img/style/block3_conv1.png')

In [ ]:
block4_conv1 = reconstruct_style(
    style_path='img/styles/starry_night_google.jpg',
    layer_names=['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1'],
    image_shape=(500, 300),
    optimize=L_BFGS(),
    iterations=10,
    callback=build_callback('build/style/block4_conv1')
)
block4_conv1.save('tex/img/style/block4_conv1.png')

In [ ]:
block5_conv1 = reconstruct_style(
    style_path='img/styles/starry_night_google.jpg',
    layer_names=['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1'],
    image_shape=(500, 300),
    optimize=L_BFGS(),
    iterations=10,
    callback=build_callback('build/style/block5_conv1')
)
block5_conv1.save('tex/img/style/block5_conv1.png')